#### We are trying to modify the power plants of this grid to obtain this energy mix:
<img src="images/target_em.png" width="30%" height="30%">

In [ ]:
import numpy as np
import os
import pandas as pd
import grid2op
from grid2op.Chronics import ChangeNothing
from chronix2grid.kpi.Generator_parameter_checker import EnergyMix_AprioriChecker
import plotly
import plotly.graph_objects as go
from grid2op.PlotGrid import NUKE_COLOR, THERMAL_COLOR, WIND_COLOR, SOLAR_COLOR, HYDRO_COLOR

#### We load the original prods_charac.csv

In [ ]:
env_name = "case118_l2rpn_wcci_benjamin"
path_ref = os.path.join("..", "example", "input", "generation")
path_tmp = os.path.join("..", "example", "custom", "input", "generation")
input_path = os.path.join(path_ref, env_name)
df = pd.read_csv(os.path.join(input_path, "prods_charac.csv"))

avg_pmaxs = df.groupby(["type"])["Pmax"].mean()
types = avg_pmaxs.index.to_numpy()
avg_pmaxs = avg_pmaxs.to_numpy()

In [ ]:
df

In [ ]:
df["type"].value_counts()

In [ ]:
def get_info_by_type(info):
  res = []
  for t in types:
    res.append(df[df["type"] == t].iloc[0][info])
  return np.array(res)

#### We recover some informations about each power plant type

In [ ]:
df = pd.read_csv(os.path.join(input_path, "prods_charac.csv"))


n = df.shape[0]
pmaxs = [250.0, 400.0, 74.7, 200.0, 67.2]
max_ramp_up = get_info_by_type("max_ramp_up")
max_ramp_down = get_info_by_type("max_ramp_down")
min_up_time = get_info_by_type("min_up_time")
min_down_time = get_info_by_type("min_down_time")
marginal_cost = get_info_by_type("marginal_cost")
shut_down_cost = get_info_by_type("shut_down_cost")
start_cost = get_info_by_type("start_cost")

In [ ]:
df["type"].value_counts()

#### We load the solution returned by the solver and modify the original csv according to the solution

YOUR "SOLVER" DO NOT WORK AT ALL !

In [ ]:
# file = open("optimization/solver/build/result.txt")
# for i in range(n):
#   idx = int(file.readline())
#   df.at[i, "type"] = types[idx]
#   df.at[i, "Pmax"] = pmaxs[idx]
#   df.at[i, "max_ramp_up"] = max_ramp_up[idx]
#   df.at[i, "max_ramp_down"] = max_ramp_down[idx]
#   df.at[i, "min_up_time"] = min_up_time[idx]
#   df.at[i, "min_down_time"] = min_down_time[idx]
#   df.at[i, "marginal_cost"] = marginal_cost[idx]
#   df.at[i, "shut_down_cost"] = shut_down_cost[idx]
#   df.at[i, "start_cost"] = start_cost[idx]

In [ ]:
# df

#### There is much more wind power plants

In [ ]:
df["type"].value_counts()

In [ ]:
output_path = os.path.join(path_tmp, env_name)
df.to_csv(os.path.join(output_path, "prods_charac.csv"), index=False)

In [ ]:
capacity_factor = np.array([30, 95, 15, np.nan, 25])
average_load = 2800
capacity_factor_df = pd.DataFrame(data=capacity_factor, columns=['capacity_factor'], index=types)

grid_path = os.path.join(output_path, "grid.json")
env118_withoutchron = grid2op.make(
    output_path,
    test=True,
    grid_path=grid_path, # assign it the 118 grid
    chronics_class=ChangeNothing, # tell it to change nothing (not the most usable environment...)
)

Target_EM_percentage=pd.DataFrame(data=[9, 36, 17, 2, 36], columns=['target_energy_mix'], index=types)

#Variable used to anticipate the energy mix a priori. Update them after chronics generation if too different
PeakLoad = 4200 #expected peak load

EnergyMix_AprioriChecker(env118_withoutchron, Target_EM_percentage, PeakLoad, average_load, capacity_factor_df)

#### You can see that the difference between the target energy mix and the apriori energy mix is 45%

### We now try to generate the loads / renewable to make sure the generated data are compatible with the grid

In [ ]:
### CONSTANT

notebook_folder=%pwd

# define your input folder
INPUT_FOLDER = os.path.join(notebook_folder, '..', 'example', 'custom', 'input')

OUTPUT_FOLDER = os.path.join(notebook_folder, '..', 'example', 'custom', 'output')

# Detailed configuration to be set in <INPUT_FOLDER>/<CASE>/params.json
start_date = "2012-01-01"
weeks = 4
n_scenarios = 1
by_n_weeks = 4

mode = 'RLTK'
mode = 'RL'


CASE = env_name

load_seed = 7
renewable_seed = 12

In [ ]:
cli_chronix2grid="chronix2grid \
                         --mode {} --output-folder {} --input-folder {} --ignore-warnings \
                         --weeks {} --case {} --n_scenarios {} --start-date {} --by-n-weeks {} \
                          --seed-for-loads {}  --seed-for-res {}".format(
                         mode, OUTPUT_FOLDER, INPUT_FOLDER, weeks, CASE, n_scenarios, start_date,by_n_weeks,
                         load_seed, renewable_seed)
cli_chronix2grid

#### But it doesn't converge

In [ ]:
!$cli_chronix2grid

In [ ]:
path_data_generated = os.path.join(OUTPUT_FOLDER, "generation", CASE, start_date, "Scenario_0")
loads_p = pd.read_csv(os.path.join(path_data_generated, "load_p.csv.bz2"), sep=";")
prods_p_renewable = pd.read_csv(os.path.join(path_data_generated, "prod_p.csv.bz2"), sep=";")

In [ ]:
is_gen_solar = env118_withoutchron.gen_type == "solar"
is_gen_wind = env118_withoutchron.gen_type == "wind"
gen_solar_name = env118_withoutchron.name_gen[is_gen_solar]
gen_wind_name = env118_withoutchron.name_gen[is_gen_wind]

In [ ]:
residual_load = loads_p.sum(axis=1) - prods_p_renewable.sum(axis=1)
proportion_solar_wind = pd.DataFrame({"total_load": loads_p.sum(axis=1),
                                      "total_solar": prods_p_renewable[gen_solar_name].sum(axis=1),
                                      "total_wind": prods_p_renewable[gen_wind_name].sum(axis=1),
                                     })

In [ ]:
min_gen_possible = np.sum(env118_withoutchron.gen_pmin[env118_withoutchron.gen_redispatchable])
max_gen_possible = np.sum(env118_withoutchron.gen_pmax[env118_withoutchron.gen_redispatchable])
max_gen_up_possible = np.sum(env118_withoutchron.gen_max_ramp_up[env118_withoutchron.gen_redispatchable])
max_gen_down_possible = np.sum(env118_withoutchron.gen_max_ramp_down[env118_withoutchron.gen_redispatchable])
loss_ratio = 0.95  # takes into account some loss (conservative)

### Share of renewables

In [ ]:
labels = ['solar','wind','unknown']
values = [proportion_solar_wind["total_solar"].sum(),
          proportion_solar_wind["total_wind"].sum(),
          proportion_solar_wind["total_load"].sum() - proportion_solar_wind["total_solar"].sum() - proportion_solar_wind["total_wind"].sum()
         ]

fig = go.Figure(data=[go.Pie(labels=labels,
                             values=values,
                             marker_colors=[SOLAR_COLOR, WIND_COLOR, 'rgba(0,0,0, 0.05)'],
                             text=[f"{round(el / 12., -3):,.0f} MWh" for el in values])
                     ]
               )
fig.update_layout(
    title=f"This has only run on {weeks} weeks, the target is for a whole year !!!"
)
fig.show()


### Basic infeasibilities

The section bellow check that the market dispatch has a chance to converge. If something is violated here, 
then there is now way the market dispatch can work

In [ ]:
due_pmin = residual_load < min_gen_possible
due_pmax = residual_load > loss_ratio * max_gen_possible
delta_gen = residual_load.diff()  # diff(t) = residual_load(t) - residual_load(t-1)
due_rampmax = delta_gen > max_gen_up_possible
due_rampmin = delta_gen < -max_gen_up_possible
print(f"There are (at least) {due_pmin.sum()} infeasibilities due to pmin ({100. * due_pmin.sum() / due_pmin.shape[0]:.0f}%)")
print(f"There are (at least) {due_pmax.sum()} infeasibilities due to pmax ({100. * due_pmax.sum() / due_pmax.shape[0]:.0f}%)")
print(f"There are (at least) {due_rampmax.sum()} infeasibilities due to ramp up ({100. * due_rampmax.sum() / due_rampmax.shape[0]:.0f}%)")
print(f"There are (at least) {due_rampmin.sum()} infeasibilities due to ramp down ({100. * due_rampmin.sum() / due_rampmin.shape[0]:.0f}%)")
assert np.all(~due_pmin) and np.all(~due_pmax) and np.all(~due_rampmax) and np.all(~due_rampmin)

### check config working

In [ ]:
all_solar_above_pmax = prods_p_renewable[gen_solar_name] > env118_withoutchron.gen_pmax[is_gen_solar]
all_solar_below_pmin = prods_p_renewable[gen_solar_name] < env118_withoutchron.gen_pmin[is_gen_solar]

all_wind_above_pmax = prods_p_renewable[gen_wind_name] > env118_withoutchron.gen_pmax[is_gen_wind]
all_wind_below_pmin = prods_p_renewable[gen_wind_name] < env118_withoutchron.gen_pmin[is_gen_wind]

assert np.all(all_solar_above_pmax.sum() == 0), f"some solar are above pmax:\n{all_solar_above_pmax.sum()}"
assert np.all(all_solar_below_pmin.sum() == 0), f"some solar are below pmin:\n{all_solar_below_pmin.sum()}"
assert np.all(all_wind_above_pmax.sum() == 0), f"some wind are above pmax:\n{all_wind_above_pmax.sum()})"
assert np.all(all_wind_below_pmin.sum() == 0), f"some wind are below pmin:\n{all_wind_below_pmin.sum()}"

## Now start the redispatching

In [ ]:
MODE_DISPATCH = "RLDT"
seed_dispatch = 0
cli_chronix2grid2="chronix2grid \
                         --mode {} --output-folder {} --input-folder {} --ignore-warnings \
                         --weeks {} --case {} --n_scenarios {} --start-date {} --by-n-weeks {} \
                          --seed-for-loads {}  --seed-for-res {} --seed-for-dispatch {}".format(
                         MODE_DISPATCH, OUTPUT_FOLDER, INPUT_FOLDER, weeks, CASE, n_scenarios, start_date,by_n_weeks,
                         load_seed, renewable_seed, seed_dispatch)
cli_chronix2grid2

In [ ]:
!$cli_chronix2grid2

In [ ]:
prods_p_total = pd.read_csv(os.path.join(path_data_generated, "prod_p.csv.bz2"), sep=";")
assert prods_p_total.shape[1] == env118_withoutchron.n_gen
loads_p2 = pd.read_csv(os.path.join(path_data_generated, "load_p.csv.bz2"), sep=";")

In [ ]:
gen_solar_name2 = env118_withoutchron.name_gen[env118_withoutchron.gen_type == "solar"]
gen_wind_name2 = env118_withoutchron.name_gen[env118_withoutchron.gen_type == "wind"]
gen_hydro_name2 = env118_withoutchron.name_gen[env118_withoutchron.gen_type == "hydro"]
gen_nuclear_name2 = env118_withoutchron.name_gen[env118_withoutchron.gen_type == "nuclear"]
gen_thermal_name2 = env118_withoutchron.name_gen[env118_withoutchron.gen_type == "thermal"]
residual_load = loads_p2.sum(axis=1) - prods_p_total.sum(axis=1)
proportion_solar_wind2 = pd.DataFrame({"total_load": loads_p2.sum(axis=1),
                                      "total_solar": prods_p_total[gen_solar_name2].sum(axis=1),
                                      "total_wind": prods_p_total[gen_wind_name2].sum(axis=1),
                                      "total_hydro": prods_p_total[gen_hydro_name2].sum(axis=1),
                                      "total_nuclear": prods_p_total[gen_nuclear_name2].sum(axis=1),
                                      "total_thermal": prods_p_total[gen_thermal_name2].sum(axis=1),
                                     })

In [ ]:
labels2 = ['solar','wind','hydro', "nuclear", "thermal"]
values2 = [proportion_solar_wind2["total_solar"].sum(),
           proportion_solar_wind2["total_wind"].sum(),
           proportion_solar_wind2["total_hydro"].sum(),
           proportion_solar_wind2["total_nuclear"].sum(),
           proportion_solar_wind2["total_thermal"].sum(),
          ]

fig = go.Figure(data=[go.Pie(labels=labels2,
                             values=values2,
                             marker_colors=[SOLAR_COLOR, WIND_COLOR, HYDRO_COLOR, NUKE_COLOR, THERMAL_COLOR],
                            text=[f"{round(el / 12., -3):,.0f} MWh" for el in values2]
                            )]
                             
                )
fig.update_layout(
    title=f"This has only run on {weeks} weeks, the target is for a whole year !!!"
)
fig.show()

### Check possible bugs

#### for renewable generators

In [ ]:
all_solar_above_pmax = prods_p_total[gen_solar_name] > env118_withoutchron.gen_pmax[is_gen_solar]
all_solar_below_pmin = prods_p_total[gen_solar_name] < env118_withoutchron.gen_pmin[is_gen_solar]

all_wind_above_pmax = prods_p_total[gen_wind_name] > env118_withoutchron.gen_pmax[is_gen_wind]
all_wind_below_pmin = prods_p_total[gen_wind_name] < env118_withoutchron.gen_pmin[is_gen_wind]

assert np.all(all_solar_above_pmax.sum() == 0), f"some solar are above pmax:\n{all_solar_above_pmax.sum()}"
assert np.all(all_solar_below_pmin.sum() == 0), f"some solar are below pmin:\n{all_solar_below_pmin.sum()}"
assert np.all(all_wind_above_pmax.sum() == 0), f"some wind are above pmax:\n{all_wind_above_pmax.sum()})"
assert np.all(all_wind_below_pmin.sum() == 0), f"some wind are below pmin:\n{all_wind_below_pmin.sum()}"


#### for the others

In [ ]:
gen_name = gen_hydro_name2

def check_controlable_gens(gen_name, env, prods_p_total):
    pmax_ = np.array([env.gen_pmax[np.where(env.name_gen == nm_)[0]] for nm_ in gen_name])
    pmax_ = pmax_.ravel()
    check_pmax = prods_p_total[gen_name] > pmax_

    pmin_ = np.array([env.gen_pmin[np.where(env.name_gen == nm_)[0]] for nm_ in gen_name])
    pmin_ = pmin_.ravel()
    check_pmin = prods_p_total[gen_name] < pmin_

    max_up_ = np.array([env.gen_max_ramp_up[np.where(env.name_gen == nm_)[0]] for nm_ in gen_name])
    max_up_ = max_up_.ravel()
    max_down_ = np.array([env.gen_max_ramp_down[np.where(env.name_gen == nm_)[0]] for nm_ in gen_name])
    max_down_ = max_down_.ravel()
    delta_gen_ = prods_p_total[gen_name].diff()  # prods_p_total.diff(t) = prods_p_total(t) - prods_p_total(t-1)
    check_max_up = delta_gen_ > max_up_
    check_max_down = delta_gen_ < -max_down_
    return check_pmax, check_pmin, check_max_up, check_max_down

check_pmax_hydro, check_pmin_hydro, check_max_up_hydro, check_max_down_hydro = check_controlable_gens(
    gen_hydro_name2, env118_withoutchron, prods_p_total)
check_pmax_nuclear, check_pmin_nuclear, check_max_up_nuclear, check_max_down_nuclear = check_controlable_gens(
    gen_nuclear_name2, env118_withoutchron, prods_p_total)
check_pmax_thermal, check_pmin_thermal, check_max_up_thermal, check_max_down_thermal = check_controlable_gens(
    gen_thermal_name2, env118_withoutchron, prods_p_total)

assert np.all(check_pmax_hydro.sum() == 0), f"some hydro are above pmax:\n{all_solar_above_pmax.sum()}"
assert np.all(check_pmin_hydro.sum() == 0), f"some hydro are below pmin:\n{all_solar_below_pmin.sum()}"
assert np.all(check_max_up_hydro.sum() == 0), f"some hydro are above max_up:\n{check_max_up_hydro.sum()}"
assert np.all(check_max_down_hydro.sum() == 0), f"some hydro are below max_down:\n{check_max_down_hydro.sum()}"

assert np.all(check_pmax_nuclear.sum() == 0), f"some nuclear are above pmax:\n{check_pmax_nuclear.sum()}"
assert np.all(check_pmin_nuclear.sum() == 0), f"some nuclear are below pmin:\n{check_pmin_nuclear.sum()}"
assert np.all(check_max_up_nuclear.sum() == 0), f"some nuclear are above max_up:\n{check_max_up_nuclear.sum()}"
assert np.all(check_max_down_nuclear.sum() == 0), f"some nuclear are below max_down:\n{check_max_down_nuclear.sum()}"

assert np.all(check_pmax_thermal.sum() == 0), f"some thermal are above pmax:\n{check_pmax_thermal.sum()}"
assert np.all(check_pmin_thermal.sum() == 0), f"some thermal are below pmin:\n{check_pmin_thermal.sum()}"
assert np.all(check_max_up_thermal.sum() == 0), f"some thermal are above max_up:\n{check_max_up_thermal.sum()}"
assert np.all(check_max_down_thermal.sum() == 0), f"some thermal are below max_down:\n{check_max_down_thermal.sum()}"

print("All checks passed !\n")

print(f"It's now time to look at the 'KPI' and the productions / loads generated at :\n\t{path_data_generated}")